In [1]:
import pandas as pd
from FastFitClassifier import FastFitClassifier
from tqdm.auto import tqdm

tqdm.pandas()

In [2]:
classifier_type = 'relevance'

model_path = "./models/relevance/avsolatorio/GIST-small-Embedding-v0"
# model_path = './models/relevance/avsolatorio/GIST-Embedding-v0'

In [3]:
def import_labelled_data(path="data/labelled/data.json"):
    data = pd.read_json(path, encoding="latin-1")
    return data



print("Loading data...")

test_dataset = import_labelled_data(path="../../data/labelled/irrelevant.json", )


print("Data loaded.")

test_dataset = test_dataset.sample(500)

test_dataset = test_dataset.sample(frac=1).reset_index(drop=True)


test_dataset.info()

Loading data...
Data loaded.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   url           500 non-null    object
 1   text          500 non-null    object
 2   relevance     500 non-null    object
 3   multiclasses  500 non-null    object
dtypes: object(4)
memory usage: 15.8+ KB


In [4]:
test_dataset

,url,text,relevance,multiclasses
0,https://documents1.worldbank.org/curated/en/92...,d e z i r o h t u A e r u s o l c DANUBE s i D...,irrelevant,[]
1,https://cabatuan.gov.ph/wp-content/uploads/202...,P HILIPPINE B IDDING D OCUMENTS (As Harmonized...,irrelevant,[]
2,https://deepdivestocks.com/wp-content/uploads/...,Liquidity Dependent Price Stability Primer DIS...,irrelevant,[]
3,https://budget.finance.go.ug/sites/default/fil...,LG Draft Budget Estimates 2024/25 VOTE: 836 Ig...,irrelevant,[]
4,https://www.decaturedc.com/wp-content/uploads/...,CONTACT Kathy Bodnar Associate E: kathy@caspia...,irrelevant,[]
...,...,...,...,...
495,https://decra.com/hubfs/Estimating-Sheet-Villa...,Estimating Sheet (Metric) - DECRA Villa Tile -...,irrelevant,[]
496,https://documents1.worldbank.org/curated/en/42...,d e z i r o h t u A e r u s o l c s i D c i l ...,irrelevant,[]
497,https://www.yourreturn.org/Treatments/Teeth/Cu...,A Special Note for Book Preview Readers Fast p...,irrelevant,[]
498,https://zenodo.org/records/10964254/files/FOSC...,1 Including traditional ecological knowledge (...,irrelevant,[]


In [5]:
import torch
torch.cuda.empty_cache()

In [6]:
classifier = FastFitClassifier(
    model_path=model_path,
    text_overlap_proportion=0.2,
    device='cuda:0'
)

Loading model from ./models/relevance/avsolatorio/GIST-small-Embedding-v0 ...
Model loaded.
Tokenizer loaded....


The model 'FastFit' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GemmaForSequenceClassification', 'Gemma2ForSequenceClassif


Classifier pipeline built.


In [7]:
import torch
torch.cuda.empty_cache()

In [8]:

#! on fastfit/modeling.py, line 838, inserting **kwargs
#! it's easier than getting the tokenizer to stop outputting
#! extra info like token_type_ids

classifier.evaluate(
    test_dataset,
    metrics=[
        "accuracy",
        "precision",
        "classification-report",
        "specificity",
        "confusion-matrix-mpl",
    ],
    aggregate="majority",
)

  0%|          | 0/500 [00:00<?, ?it/s]

Data chunked.


Files	:   0%|          | 0/500 [00:00<?, ?it/s]

Chunks	:   0%|          | 0/159248 [00:00<?, ?it/s]

In [ ]:
t = test_dataset.sample(10)
classifier.predict(t)

  0%|          | 0/10 [00:00<?, ?it/s]

Data chunked.


Files	:   0%|          | 0/10 [00:00<?, ?it/s]

Chunks	:   0%|          | 0/3343 [00:00<?, ?it/s]

Calculated predictions.


,url,text,relevance,multiclasses,predictions
350,https://repositorio.uncp.edu.pe/bitstream/hand...,Facultad de Sociología Reasentamiento e implic...,irrelevant,[],irrelevant
1861,https://www.arla.ph/49f584/globalassets/arla-g...,20 årsredovisning 12 innehåll FÖrvAlTningsrAPP...,irrelevant,[],irrelevant
812,https://www.daywireless.com/downloads/wave/eme...,PRIORITY COMMUNICATION FOR BROADBAND PTT | BRO...,irrelevant,[],irrelevant
1567,https://redge.org.pe/sites/default/files/La%20...,La otra lucha contra el cáncer Justicia Fiscal...,irrelevant,[],irrelevant
353,https://www.ddmworks.com/assets/images/instruc...,Solstice /Sky Short Throw Shifter This short t...,irrelevant,[],irrelevant
547,https://gatt-disputes.wto.org/sites/default/fi...,"RESTRICTED 25 GENERAL AGREEMENT O N %V , * Q i...",irrelevant,[],irrelevant
93,https://budget.finance.go.ug/sites/default/fil...,LG Draft Budget Estimates 2024/25 VOTE: 823 Bu...,irrelevant,[],irrelevant
1596,https://fami.com.ph/wp-content/uploads/2024/05...,COVER SHEET C S 2 0 1 8 - 0 1 6 4 6 SEC Regist...,irrelevant,[],irrelevant
1586,https://documents1.worldbank.org/curated/en/09...,d e z i r o h t u A e r u s o l c s i D c i l ...,irrelevant,[],irrelevant
1138,https://documents1.worldbank.org/curated/en/77...,d e z i r o h t u A e r u s o l c s i D c i l ...,irrelevant,[],irrelevant


In [ ]:
t = pd.DataFrame(
    [
        {"text": "Quantifying the effects of Entry Level Stewardship (ELS) on biodiversity at the farm scale: The Hillesden Experiment", "relevance": "relevant"},
        {"text": "The economy in the UK depends on a number of factors: the government", "relevance": "irrelevant"},
    ]
)
classifier.predict(t)

  0%|          | 0/2 [00:00<?, ?it/s]

Data chunked.


Files	:   0%|          | 0/2 [00:00<?, ?it/s]

Chunks	:   0%|          | 0/2 [00:00<?, ?it/s]

Calculated predictions.


,text,relevance,predictions
0,Quantifying the effects of Entry Level Steward...,relevant,relevant
1,The economy in the UK depends on a number of f...,irrelevant,irrelevant
